In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/image_course

/content/drive/MyDrive/image_course


In [ ]:
import os
import re

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input,RNN,GRU, Embedding, Attention, Concatenate,Dropout,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
import random

keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
with open('Dataset.txt', 'r') as file:
  text = file.readlines()
  lines = [line.strip() for line in text]
  input_texts, output_texts = [], []
  lines = lines[:10000]
  random.shuffle(lines)
  for l in lines:
    input_text = l.split('\t')[0]
    output_text = l.split('\t')[1]
    #input_texts.append(input_text)
    #output_texts.append(output_text)
    input_texts.append('<' + input_text + '>')
    output_texts.append('<' + output_text + '>')

(40, 10000)

In [ ]:
input_characters = set()

for sentence in input_texts:
  for char in sentence:
    input_characters.add(char)

In [ ]:
output_characters = set()

for sentence in output_texts:
  for char in sentence:
    output_characters.add(char)

In [ ]:
len(input_characters), len(output_characters)

(75, 107)

In [ ]:
input_characters = sorted(list(input_characters))
output_characters = sorted(list(output_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(output_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in output_texts])

num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length

(75, 107, 46, 67)

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
output_token_index = dict([(char, i) for i, char in enumerate(output_characters)])

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '<': 18,
 '>': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'X': 44,
 'Y': 45,
 'Z': 46,
 'a': 47,
 'b': 48,
 'c': 49,
 'd': 50,
 'e': 51,
 'f': 52,
 'g': 53,
 'h': 54,
 'i': 55,
 'j': 56,
 'k': 57,
 'l': 58,
 'm': 59,
 'n': 60,
 'o': 61,
 'p': 62,
 'q': 63,
 'r': 64,
 's': 65,
 't': 66,
 'u': 67,
 'v': 68,
 'w': 69,
 'x': 70,
 'y': 71,
 'z': 72,
 '’': 73,
 '€': 74}

In [ ]:
output_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '/': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '<': 20,
 '>': 21,
 '?': 22,
 'F': 23,
 'I': 24,
 'M': 25,
 'P': 26,
 'T': 27,
 '\\': 28,
 'a': 29,
 'b': 30,
 'd': 31,
 'e': 32,
 'f': 33,
 'g': 34,
 'h': 35,
 'i': 36,
 'l': 37,
 'n': 38,
 'o': 39,
 'r': 40,
 's': 41,
 't': 42,
 'x': 43,
 'y': 44,
 '،': 45,
 '؟': 46,
 'ء': 47,
 'آ': 48,
 'أ': 49,
 'ؤ': 50,
 'إ': 51,
 'ئ': 52,
 'ا': 53,
 'ب': 54,
 'ة': 55,
 'ت': 56,
 'ث': 57,
 'ج': 58,
 'ح': 59,
 'خ': 60,
 'د': 61,
 'ذ': 62,
 'ر': 63,
 'ز': 64,
 'س': 65,
 'ش': 66,
 'ص': 67,
 'ض': 68,
 'ط': 69,
 'ظ': 70,
 'ع': 71,
 'غ': 72,
 'ف': 73,
 'ق': 74,
 'ك': 75,
 'ل': 76,
 'م': 77,
 'ن': 78,
 'ه': 79,
 'و': 80,
 'ى': 81,
 'ي': 82,
 'ً': 83,
 'ٌ': 84,
 'ٍ': 85,
 'َ': 86,
 'ُ': 87,
 'ِ': 88,
 'ّ': 89,
 'ْ': 90,
 '٠': 91,
 '١': 92,
 '٢': 93,
 '٣': 94,
 '٤': 95,
 '٥': 96,
 '٦': 97,
 '٨': 98,
 '٩': 99,
 'ٱ': 100

In [ ]:
num_samples = 10000

In [ ]:
# Input Sequences (English): Padded to a maximum length of 25 characters with a vocabulary of 80 different characters (50000, 25, 80).
# Output Sequences (French): Padded to a maximum length of 69 characters with a vocabulary of 103 different characters (10000, 69, 103).

encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_output_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

encoder_input_data.shape, decoder_input_data.shape

((10000, 46, 75), (10000, 67, 107))

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, output_texts)):
  for t, char in enumerate (input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.0

  encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.0

  for t, char in enumerate(target_text):
    decoder_input_data[i, t, output_token_index[char]] = 1.0
    if t > 0:
      decoder_output_data[i, t - 1, output_token_index[char]] = 1.0

  decoder_input_data[i, t + 1:, output_token_index[' ']] = 1.0
  decoder_output_data[i, t:, output_token_index[' ']] = 1.0

In [ ]:
encoder_input_data.shape

(10000, 46, 75)

In [ ]:
encoder_input_data[0].shape

(46, 75)

In [ ]:
encoder_input_data[0][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
encoder_input_data[0][2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [ ]:
lstm_dim = 16

# Encoder Definition

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = Bidirectional(LSTM(lstm_dim, return_sequences=False, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]


In [ ]:
# Decoder Definiton

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(lstm_dim * 2, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

In [ ]:
# Dense Layer
dense_hidden = Dense(128, activation='relu')  # Added hidden dense layer
dropout = Dropout(0.3)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
dense_outputs = dense_hidden(decoder_lstm_outputs)
dense_outputs = dropout(dense_outputs)
decoder_outputs = decoder_dense(decoder_lstm_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x=[encoder_input_data, decoder_input_data], y=decoder_output_data, batch_size=128, epochs=50, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)])

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - accuracy: 0.4021 - loss: 4.2282 - val_accuracy: 0.7235 - val_loss: 1.6746
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.7235 - loss: 1.5482 - val_accuracy: 0.7235 - val_loss: 1.3100
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7235 - loss: 1.2721 - val_accuracy: 0.7247 - val_loss: 1.1806
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7243 - loss: 1.1577 - val_accuracy: 0.7254 - val_loss: 1.1026
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7252 - loss: 1.0906 - val_accuracy: 0.7264 - val_loss: 1.0664
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7257 - loss: 1.0569 - val_accuracy: 0.7267 - val_loss: 1.0375
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7269 - loss: 1.0321 - val_accuracy: 0.7283 - val_loss: 1.0150
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7320 - loss: 1.0101 - val_accuracy: 0.7419 - v

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 75)       │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ [(None, 32), (None,    │         11,776 │ input_layer[0][0]      │
│ (Bidirectional)           │ 16), (None, 16),       │                │                        │
│                           │ (None, 16), (None,     │                │                        │
│                           │ 16)]                   │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 107)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 32)             │              0 │ bidirectional[0][1],   │
│                           │                        │                │ bidirectional[0][3]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 32)             │              0 │ bidirectional[0][2],   │
│ (Concatenate)             │                        │                │ bidirectional[0][4]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 32),     │         17,920 │ input_layer_1[0][0],   │
│                           │ (None, 32), (None,     │                │ concatenate[0][0],     │
│                           │ 32)]                   │                │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, None, 107)      │          3,531 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 99,683 (389.39 KB)

 Trainable params: 33,227 (129.79 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 66,456 (259.60 KB)

In [ ]:
lstm_dim = 16

att_encoder_inputs = Input(shape=(None, num_encoder_tokens))

att_encoder = Bidirectional(LSTM(lstm_dim, return_sequences=True, return_state=True))
att_encoder_outputs, att_forward_h, att_forward_c, att_backward_h, att_backward_c = att_encoder(att_encoder_inputs)

att_state_h = Concatenate()([att_forward_h, att_backward_h])
att_state_c = Concatenate()([att_forward_c, att_backward_c])

att_encoder_states = [att_state_h, att_state_c]


In [ ]:
# Decoder Definition
att_decoder_inputs = Input(shape=(None, num_decoder_tokens))

att_decoder_lstm = LSTM(lstm_dim  * 2, return_sequences=True, return_state=True)
att_decoder_outputs, _, _ = att_decoder_lstm(att_decoder_inputs, initial_state=att_encoder_states)

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Input, Attention, Concatenate

attention = Attention(use_scale=True)([att_decoder_outputs, att_encoder_outputs])

att_decoder_combined = Concatenate(axis=-1)([att_decoder_outputs, attention])


In [ ]:
att_dense_hidden = Dense(128, activation='relu')
att_dropout = Dropout(0.3)
att_decoder_dense = Dense(num_decoder_tokens, activation='softmax')
att_dense_outputs = att_dense_hidden(att_decoder_combined)
att_dense_outputs = att_dropout(att_dense_outputs)
att_decoder_outputs = att_decoder_dense(att_decoder_combined)

In [ ]:
att_model = Model([att_encoder_inputs, att_decoder_inputs], att_decoder_outputs)

In [ ]:
att_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, None, 75)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ [(None, None, 32),     │         11,776 │ input_layer_2[0][0]    │
│ (Bidirectional)           │ (None, 16), (None,     │                │                        │
│                           │ 16), (None, 16),       │                │                        │
│                           │ (None, 16)]            │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_3             │ (None, None, 107)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 32)             │              0 │ bidirectional_1[0][1], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][3]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 32)             │              0 │ bidirectional_1[0][2], │
│ (Concatenate)             │                        │                │ bidirectional_1[0][4]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 32),     │         17,920 │ input_layer_3[0][0],   │
│                           │ (None, 32), (None,     │                │ concatenate_2[0][0],   │
│                           │ 32)]                   │                │ concatenate_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)     │ (None, None, 32)       │              1 │ lstm_3[0][0],          │
│                           │                        │                │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, None, 64)       │              0 │ lstm_3[0][0],          │
│ (Concatenate)             │                        │                │ attention[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, None, 107)      │          6,955 │ concatenate_4[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 36,652 (143.17 KB)

 Trainable params: 36,652 (143.17 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
att_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

att_model.fit(x=[encoder_input_data, decoder_input_data], y=decoder_output_data, batch_size=128, epochs=50, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)])

Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.6067 - loss: 3.7902 - val_accuracy: 0.7176 - val_loss: 1.3779
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7203 - loss: 1.3035 - val_accuracy: 0.7235 - val_loss: 1.1563
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7237 - loss: 1.1353 - val_accuracy: 0.7244 - val_loss: 1.0920
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7239 - loss: 1.0842 - val_accuracy: 0.7260 - val_loss: 1.0638
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7255 - loss: 1.0589 - val_accuracy: 0.7266 - val_loss: 1.0436
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7269 - loss: 1.0399 - val_accuracy: 0.7293 - val_loss: 1.0267
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7286 - loss: 1.0236 - val_accuracy: 0.7296 - val_loss: 1.0110
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7320 - loss: 1.0071 - val_accuracy: 0.7386 - v